<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/Double_q_learning_%26_DDQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gymnasium
!pip install gymnasium[atari]
!pip install gymnasium[accept-rom-license]
!pip install imageio
!pip install imageio-ffmpeg

In [ ]:
import gymnasium as gym
import imageio
from gymnasium.wrappers import FrameStack, GrayScaleObservation
import torch
from torch import nn
import numpy as np
import torch.optim as optim
import random
import torch.nn.functional as F
from google.colab import drive
from tqdm.auto import tqdm
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

drive.mount('/content/drive')

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 매개변수로 repeat을 추가.(게임 반복 횟수)

def play_game(model, env, repeat, file_name, record : bool = False):

    total_reward = 0
    frames = []

    for i in range(repeat):

        obs, info = env.reset()
        if record == True:
            frames.append(env.render())

        obs, reward, terminated, truncated, info = env.step(1)
        if record == True:
            frames.append(env.render())

        while(terminated == False and truncated == False and info['lives' == 5]):

            _, action = model.greedy_action(torch.tensor(np.array(obs)).to(DEVICE))
            obs, reward, terminated, truncated, info = env.step(action)
            if record == True:
                frames.append(env.render())

            total_reward += reward

        if record == True:
            with imageio.get_writer(f'/content/drive/MyDrive/동영상/{file_name}.mp4', fps=30, ) as video:
                for frame in frames:
                    video.append_data(frame)

        total_reward = total_reward / repeat

        return total_reward

# 모델 업데이트.

def model_update(q_model, target_model, optimizer, buffer, batch_size):

    obs_batch, action_batch, reward_batch, nobs_batch = buffer.sample(batch_size)

    obs_batch = torch.tensor(np.array(obs_batch)).float().to(DEVICE)
    action_batch = torch.tensor(np.array(action_batch)).float().to(DEVICE)
    reward_batch = torch.tensor(np.array(reward_batch)).float().to(DEVICE)
    nobs_batch = torch.tensor(np.array(nobs_batch)).float().to(DEVICE)

    with torch.no_grad():
        _, idx = q_model.greedy_action(nobs_batch) # 행동선택은 q모델의 정책에 따른다.
        y = target_model.generate_q(nobs_batch, idx) # 선택된 행동에 대한 q값 평가는 타겟모델로 진행한다.
        y = 0.99*y
        y = y + reward_batch

    q = q_model.generate_q(obs_batch, action_batch)

    loss = F.mse_loss(y, q)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss = loss.item() # .item()은 텐서가 단일값을 포함하고 있을 경우 파이썬의 수자로 변환.

    return loss

def step_and_stack(model, env, buffer, epsilon, step_size):

    obs, reward, terminated, truncated, info = env.step(1)

    for i in range(step_size):

        if terminated == False and truncated == False and info['lives'] == 5:

            action = model.epsilon_greedy_action(torch.tensor(np.array(obs)).to(DEVICE), epsilon)
            nobs, reward, terminated, truncated, info = env.step(action)

            buffer.add([obs, action, reward, nobs])
            obs = nobs

        else:

            obs, info = env.reset()
            nobs, reward, terminated, truncated, info = env.step(1)
            buffer.add([obs, 1, reward, nobs])
            obs = nobs

    return

In [ ]:
class ExperienceReplayMemory:

    def __init__(self, capacity):

        self.capacity = capacity
        self.memory = []

    def add(self, experience):

        if len(self.memory) < self.capacity:
            self.memory.append(experience)

        else:
            self.memory.pop(0) # 리스트가 클 경우 매우 비효율적. 대안으로 deque를 사용할 수 있지만 인데싱이 불편하고 느림.
            self.memory.append(experience)

    def sample(self, batch_size):

        sample = random.sample(self.memory, batch_size)

        obs_batch = [obs for obs, _, _, _ in sample]
        action_batch = [action for _, action, _, _ in sample]
        reward_batch = [reward for _, _, reward, _ in sample]
        nobs_batch = [nobs for _, _, _, nobs in sample]

        return obs_batch, action_batch, reward_batch, nobs_batch

    def __len__(self):

        return len(self.memory)

In [ ]:
class DDQN(nn.module):
    def __init__(self):

        super().__init__()

        self.conv = nn.Sequential(nn.Conv2d(4, 16, kernel_size = 8, stride = 4),
                                  nn.ReLU(),
                                  nn.Conv2d(16, 32, kernel_size = 4, stride =2),
                                  nn.ReLU())